# Preparación del Entorno

Sobre Ensambles

 ---------------------------
Step 1: Conceptos
---------------------------

 Hay quienes pasan por el bosque y sólo ven leña para el fuego
 - León Tolstoi

Preguntas
- ¿Qué es un ensamble de modelos?
- ¿Cómo tienen que ser los modelos dentro de un ensamble?
- ¿Qué técnicas conoce para ensamblar modelos?
- ¿Por qué funcionan mejor los ensambles?


Los ensambles pueden partir de modelos ya desarrollados, o de modelos que se creen especialmente para ser ensamblados.

 Sobre los segundos, veremos son los llamados Bagging (bootstrap aggregating).
 - Hacer **N** nuevos conjunto de entrenamiento usando boostraping, o sea,
   reemplazar nuestro dataset por elementos aleatorios con reemplazo.
 - Para este cada nuevo dataset obtener un modelo.
 - Promediar (o votar) las salidas de los modelos.

El espíritu detrás de este algoritmo, puede entenderse en que cada modelo es una especialista de sólo una parte, y la suma de muchos especialistas consiguen un buen modelo.

- El algoritmo de **bagging** más conocido es el **random fore

## Importación de Librerías

In [1]:
# Aplicacion de modelo

# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,627623,33.6,1404614,75.1,994946,53.2
Vcells,1149764,8.8,8388608,64.0,1769393,13.5


In [2]:
# Librerías necesarias
require("data.table")
require("rpart")
require("ggplot2")
require("ranger")
require("randomForest")
require("lightgbm")

Loading required package: data.table



Loading required package: rpart

Loading required package: ggplot2

Loading required package: ranger

Loading required package: randomForest

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ranger':

    importance


The following object is masked from 'package:ggplot2':

    margin


Loading required package: lightgbm

Loading required package: R6



In [3]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 1") # Establezco el Working Directory

# cargo el dataset
dataset <- fread("competencia_01_alternativo.csv")

In [4]:
# Poner sus semillas
semillas <- c(279511, 279523, 279541, 279551, 279571)

In [5]:
# Nos quedamos solo con el 202103
dataset <- dataset[foto_mes == 202103]

# Step 2: Primer Random Forest

In [6]:
# Importante que la clase sea factor
dataset[, clase_binaria1 := factor(ifelse(clase_ternaria == "BAJA+2", "evento", "noevento"))]

In [7]:
dataset$clase_ternaria <- NULL
in_training <- caret::createDataPartition(dataset$clase_binaria1,
                     p = 0.70, list = FALSE)

dtrain  <-  dataset[in_training, ]
dtest   <-  dataset[-in_training, ]

# ranger no soporta, como lo hacen otras librerías, los missing values
dtrain <-  na.roughfix(dtrain)
dtest <-  na.roughfix(dtest)

# Cantidad de variables que abren por cada hoja
n_variables <- round(sqrt(dim(dtrain)[2] - 1))


In [8]:
t0 <- Sys.time()
modelo_rf_1 <- ranger(clase_binaria1 ~ ., data = dtrain,
                  probability = TRUE,
                  num.trees = 100,
                  min.node.size=10,  # <---------
                  mtry = n_variables,
                  splitrule = "gini",
                  sample.fraction = 0.66,
                  importance = "impurity",
                  verbose = TRUE)
t1 <- Sys.time()
as.numeric(t1 - t0, units = "secs")

[1] 20.18665

# Step 3: Midiendo el primero RF

In [9]:
pred_train <- predict(modelo_rf_1, dtrain)
pred_test <- predict(modelo_rf_1, dtest)

# Ganancia en dtrain
print(sum((pred_train$predictions[, "evento"] >= 0.025) * ifelse(
                    dtrain$clase_binaria1 == "evento", 
                    273000, -7000) / 0.7))
# Ganancia en dtest
print(sum((pred_test$predictions[, "evento"] >= 0.025) * ifelse(
                    dtest$clase_binaria1 == "evento",
                    273000, -7000) / 0.3))

[1] 214500000
[1] 60270000


Preguntas
- ¿Qué paso en `train`?
- ¿Se veía esa diferencia en los árboles?


# Step 4: Importancia de variables

In [10]:
importancia <- as.data.table(modelo_rf_1$variable.importance,
                    keep.rownames = TRUE)
colnames(importancia) <- c("variable", "importancia")
setorder(importancia, -importancia)
importancia

variable,importancia
<chr>,<dbl>
mcuentas_saldo,18.963521
mrentabilidad_annual,18.175787
mpasivos_margen,18.057882
numero_de_cliente,17.041501
ctrx_quarter,16.408563
mcuenta_corriente,15.824942
cliente_edad,15.586140
cliente_antiguedad,15.406579
mactivos_margen,15.217435


Preguntas
 - ¿Qué significa que una variable sea más importante que otra?
 - ¿Qué significa que una variable tenga 0 importancia?
 - ¿Con el **RF** es suficiente como para descartarlas?
 - ¿Qué una variable tenga algo de importancia es suficiente como para
 - entender que da valor?


# Step 5: Un experimento con canaritos

In [11]:
dtrain$canarito <- runif(nrow(dtrain))

modelo_rf_2 <- ranger(clase_binaria1 ~ ., data = dtrain,
                  probability = TRUE,
                  num.trees = 150,
                  min.node.size = 10, # <-- probar con valores mas altos
                  mtry = n_variables,
                  splitrule = "gini",
                  importance = "impurity",
                  verbose = TRUE)

importancia2 <- as.data.table(modelo_rf_2$variable.importance,
                    keep.rownames = TRUE)
colnames(importancia2) <- c("variable", "importancia")
setorder(importancia2, -importancia)
importancia2
which(importancia2$variable == "canarito")

## Active learning o a llorar a la iglesia.

Growing trees.. Progress: 81%. Estimated remaining time: 7 seconds.


variable,importancia
<chr>,<dbl>
mcuentas_saldo,30.496066
mrentabilidad_annual,28.045695
mpasivos_margen,27.752238
numero_de_cliente,27.129295
canarito,26.940222
ctrx_quarter,25.472924
cliente_edad,25.365977
mcuenta_corriente,25.049533
mrentabilidad,23.892803


[1] 5

In [12]:
## ---------------------------
## Step 5.1: Hablando de los Extra Trees
## ---------------------------

modelo_rf_3 <- ranger(clase_binaria1 ~ ., data = dtrain,
                  probability = TRUE,
                  num.trees = 150,
                  min.node.size = 200, # <---------
                  mtry = n_variables,
                  splitrule = "extratrees", # <---------
                  num.random.splits = 10, # <---------
                  importance = "impurity",
                  verbose = TRUE)

importancia3 <- as.data.table(modelo_rf_3$variable.importance,
                    keep.rownames = TRUE)
colnames(importancia3) <- c("variable", "importancia")
setorder(importancia3, -importancia)
importancia3
which(importancia3$variable == "canarito")

Growing trees.. Progress: 76%. Estimated remaining time: 9 seconds.


variable,importancia
<chr>,<dbl>
ctrx_quarter,7.521274
active_quarter,5.433602
ccajas_consultas,5.200680
cproductos,4.703825
cliente_edad,3.474520
mcomisiones_mantenimiento,3.281442
mcuenta_corriente,3.218174
ccomisiones_mantenimiento,2.987467
ccomisiones_otras,2.979031


[1] 20

# Step 6: Boosting, la navaja suiza de los modelos - Conceptos

 Estos se construyen de forma serial.
- Primero se parte de un modelo (que puede ser un valor constante) y se  complementa con un modelo que busca mejorar al anterior.

Hay dos algoritmos muy conocidos de este tipo:

- **Adaboost**: Que cada nuevo modelo va mejorando a los anteriores poniendo un peso mayor en los casos donde la clasificación es incorrecta

- **Gradient Boosting**: Que cada nuevo modelo va mejorando los anteriores, tratando de corregir los residuos, buscando estos últimos con el gradiente de una función de perdida.

Este último se empezó a hacer muy popular por la excelente pieza de tecnología que es su implementación **xgboost**, superado luego por el LightGBM.

# Step 7: LightGBM

In [13]:
# Cargamos todo para tener un código limpio
dataset <- fread("competencia_01_alternativo.csv")

# Nos quedamos solo con el 202103
dataset <- dataset[foto_mes == 202103]

clase_binaria <- ifelse(dataset$clase_ternaria == "BAJA+2", 1, 0)
dataset$clase_ternaria <- NULL

dtrain  <- lgb.Dataset(data = data.matrix(dataset), label = clase_binaria)


In [14]:
set.seed(semillas[1])
# LightGBM, al igual que XGB traen su implementación del CV
# Los parámetros los iremos viendo en profundidad la siguiente clase.
model_lgbm_cv <- lgb.cv(data = dtrain,
         eval = "auc",
         stratified = TRUE,
         nfold = 5,
         feature_pre_filter=FALSE,
         param = list(objective = "binary",
                       max_bin = 15,
                       min_data_in_leaf = 300,
                       learning_rate = 0.05
                       )
      )

# Mejor iteración
model_lgbm_cv$best_iter

# Una vez que elegimos los parámetros tenemos que entrenar con todos.
model_lgm <- lightgbm(data = dtrain,
            nrounds = model_lgbm_cv$best_iter,
            params = list(objective = "binary",
                            max_bin = 15,
                            min_data_in_leaf = 4000,
                            learning_rate = 0.05),
             verbose = -1)

# También tiene su importancia de variables
lgb.importance(model_lgm, percentage = TRUE)

## Bienvenido al mundo de los ensambles

Warning message in lgb.cv(data = dtrain, eval = "auc", stratified = TRUE, nfold = 5, :
"lgb.cv: Found the following passed through '...': feature_pre_filter. These will be used, but in future releases of lightgbm, this warning will become an error. Add these to 'params' instead. See ?lgb.cv for documentation on how to call this function."


[LightGBM] [Info] Number of positive: 780, number of negative: 129879
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049225 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1934
[LightGBM] [Info] Number of data points in the train set: 130659, number of used features: 154
[LightGBM] [Info] Number of positive: 775, number of negative: 129884
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039967 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1934
[LightGBM] [Info] Number of data points in the train set: 130659, number of used features: 154
[LightGBM] [Info] Number of positive: 782, number of negative: 129877
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead 

[1] 65

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
ctrx_quarter,0.405383000,0.1156625178,0.0685358255
cpayroll_trx,0.079414003,0.0746348214,0.0259605400
mpasivos_margen,0.046114690,0.0230858603,0.0249221184
mprestamos_personales,0.044083420,0.0313805168,0.0181723780
mpayroll,0.041305883,0.0576951109,0.0254413292
mcuentas_saldo,0.038841315,0.0259066799,0.0295950156
mcaja_ahorro,0.035607134,0.0057356225,0.0103842160
mrentabilidad_annual,0.028864194,0.0292648476,0.0446521288
mtarjeta_visa_consumo,0.023057361,0.0409682346,0.0306334372
